In [402]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [403]:
#https://archive.ics.uci.edu/ml/datasets/Japanese+Credit+Screening
data = pd.read_csv("crx.data", header=None)
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


Последняя колонка представляет собой положительние и отрицальные случаи, когда люди получали и не получали кредит \
Те 14 признаков и 1 целевая переменная

In [404]:
print(data.shape)

(690, 16)


Всего строк 690

In [405]:
data.iloc[:, -1].value_counts()

-    383
+    307
Name: 15, dtype: int64

In [406]:
data.loc[data[15] == '-', 15] = 0
data.loc[data[15] == '+', 15] = 1

In [407]:
data.iloc[:, -1].value_counts()

0    383
1    307
Name: 15, dtype: int64

In [408]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,1


###  feature engineering

Согласно описанию(crx.names) датасета, в нем присутсвуют пропуски, попробую заполнить их

In [409]:
def f(s):
    mask = s.isna()
    return np.where(mask, np.random.choice(s[~mask], len(s)), s)

In [410]:
for i in range(len(data.columns)):
    data[i].loc[data[i] == '?']
    data[i] = data[i].replace('?', np.NaN)
    data[i] = f(data[i])

In [411]:
for i in range(len(data.columns)):
    print(data[i].loc[data[i] == 'Nan'])

Series([], Name: 0, dtype: object)
Series([], Name: 1, dtype: object)
Series([], Name: 2, dtype: float64)
Series([], Name: 3, dtype: object)
Series([], Name: 4, dtype: object)
Series([], Name: 5, dtype: object)
Series([], Name: 6, dtype: object)
Series([], Name: 7, dtype: float64)
Series([], Name: 8, dtype: object)
Series([], Name: 9, dtype: object)
Series([], Name: 10, dtype: int64)
Series([], Name: 11, dtype: object)
Series([], Name: 12, dtype: object)
Series([], Name: 13, dtype: object)
Series([], Name: 14, dtype: int64)
Series([], Name: 15, dtype: int64)


In [412]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,1


In [413]:
for i in [0, 1, 3, 4, 5, 6, 8, 9, 11, 12, 13]:
    for j in range(len(data[i].value_counts())):
        data[i] = data[i].replace(data[i].value_counts().index[j], j)

In [414]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,255,0.000,0,0,2,0,1.25,0,1,1,0,0,114,0,1
1,1,308,4.460,0,0,1,1,3.04,0,1,6,0,0,164,560,1
2,1,24.50,0.500,0,0,1,1,1.50,0,0,0,0,0,6,824,1
3,0,7,1.540,0,0,2,0,3.75,0,1,5,1,0,4,3,1
4,0,62,5.625,0,0,2,0,1.71,0,0,0,0,1,2,0,1


In [423]:
data = data.drop(columns=[1, 5, 13])

### Обучение

In [424]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [425]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

[14:57:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Проверяем качество

In [426]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 84.21%
roc: 86.55%
recall: 84.21%
precision: 84.21%


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [428]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 77/307 as positives and unlabeling the rest


In [429]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    613
 1     77
Name: class_test, dtype: int64


In [430]:
mod_data.head(10)

,0,2,3,4,6,7,8,9,10,11,12,14,15,class_test
0,0,0.000,0,0,0,1.250,0,1,1,0,0,0,1,-1
1,1,4.460,0,0,1,3.040,0,1,6,0,0,560,1,1
2,1,0.500,0,0,1,1.500,0,0,0,0,0,824,1,-1
3,0,1.540,0,0,0,3.750,0,1,5,1,0,3,1,1
4,0,5.625,0,0,0,1.710,0,0,0,0,1,0,1,1
5,0,4.000,0,0,0,2.500,0,0,0,1,0,0,1,-1
6,0,1.040,0,0,1,6.500,0,0,0,1,0,31285,1,1
7,1,11.585,0,0,0,0.040,0,0,0,0,0,1349,1,1
8,0,0.500,1,1,1,3.960,0,0,0,0,0,314,1,-1
9,0,4.915,1,1,0,3.165,0,0,0,1,0,1442,1,1


In [431]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling

In [432]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(77, 14) (77, 14)


In [433]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

[15:53:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification results:
f1: 76.28%
roc: 81.02%
recall: 95.07%
precision: 63.70%


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Судя по метрикам модель, возможно, но не сильно, переобучается